## Fidelity and First Order Conditions

#### Imports

In [1]:
using Pkg
Pkg.activate(".")
# Remove and re-add the package
Pkg.rm("QuantumCollocation")
Pkg.develop(path="../../QuantumCollocation.jl")
using Revise
using QuantumCollocation
using Piccolo

  Activating project at `~/Documents/research/pulses/project/notebooks/src`
    Updating `~/Documents/research/pulses/project/notebooks/src/Project.toml`
  [0dc23a59] - QuantumCollocation v0.8.0 `../../QuantumCollocation.jl`
    Updating `~/Documents/research/pulses/project/notebooks/src/Manifest.toml`
  [47edcb42] - ADTypes v1.15.0
  [c29ec348] - AbstractDifferentiation v0.4.4
  [7d9f7c33] - Accessors v0.1.42
  [79e6a3ab] - Adapt v4.3.0
  [66dad0bd] - AliasTables v1.1.3
  [4fba245c] - ArrayInterface v7.19.0
  [4c555306] - ArrayLayouts v1.11.1
  [6e4b80f9] - BenchmarkTools v1.6.0
  [62783981] - BitTwiddlingConvenienceFunctions v0.1.6
  [70df07ce] - BracketingNonlinearSolve v1.3.0
  [2a0fbf3d] - CPUSummary v0.2.6
  [d360d2e6] - ChainRulesCore v1.25.2
  [fb6a15b2] - CloseOpenIntervals v0.1.13
  [523fee87] - CodecBzip2 v0.8.5
  [944b1d66] - CodecZlib v0.7.8
  [38540f10] - CommonSolve v0.2.4
  [bbf7d656] - CommonSubexpressions v0.3.1
  [f70d9fcc] - CommonWorldInvalidations v1.0.0
  [34da21

#### Setup

In [2]:
Hₑ = PAULIS.X

a = annihilate(3)
sys = QuantumSystem([(a + a')/2, (a - a')/(2im)])
U_goal = EmbeddedOperator(GATES[:H], sys)
T = 51
Δt = 0.2


0.2

#### Custom Objective


In [3]:
prob = UnitarySmoothPulseProblem(
            sys, U_goal, T, Δt;
            H_err=Hₑ
        )

solve!(prob, max_iter=5, verbose=false, print_level=1)

    constructing UnitarySmoothPulseProblem...
	using integrator: typeof(UnitaryIntegrator)
	control derivative names: [:da, :dda]
	applying timesteps_all_equal constraint: Δt


MethodError: MethodError: no method matching _array_for(::Vector{ForwardDiff.Dual{ForwardDiff.Tag{DirectTrajOpt.Objectives.var"#24#30"{Int64, Vector{Float64}, DirectTrajOpt.Objectives.var"#36#38"{QuantumCollocation.QuantumObjectives.var"#ℓ#21"{Matrix{ComplexF64}, NamedTrajectory{Float64, (:Ũ⃗, :a, :da, :dda, :Δt), NTuple{5, Int64}, (:a, :da, :dda, :Δt), NTuple{4, Tuple{Vector{Float64}, Vector{Float64}}}, (:Ũ⃗, :a), Tuple{Vector{Float64}, Vector{Float64}}, (:a,), Tuple{Vector{Float64}}, (:Ũ⃗,), Tuple{Vector{Float64}}, (:Ũ⃗, :a, :da, :dda, :Δt), NTuple{5, UnitRange{Int64}}, NTuple{5, Symbol}, Tuple{Symbol, Symbol, Symbol}, Tuple{Symbol, Symbol}, (), Tuple{}, (), Tuple{}, Tuple{}}}}, Vector{Nothing}}, Float64}, Float64, 9}}, ::UnitRange{Int64}, ::Base.HasShape{1})
The function `_array_for` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  _array_for(!Matched::Type{T}, ::Any, ::Any) where T
   @ Base array.jl:679
  _array_for(!Matched::Type